## DashBoard Contact Center

Notebook donde se diseña una app de Streamlit para predecir la cantidad de llamdas a recibir en base a una serie de info

In [1]:
import pandas as pd
import numpy as np
import streamlit as st
import pyodbc
import sys

from sklearn.preprocessing import OneHotEncoder
import datetime
import calendar

import pickle
from sklearn.preprocessing import StandardScaler

### Datos de conexion

In [2]:
path_pass = 'data/pass.csv'
try:
    df_pass = pd.read_csv(path_pass)
    password = df_pass.iloc[0,0]
except:
    print('No hay fichero para cargar')
    password = 'null'

# Datos Llamadas

In [3]:
def getCallData():
    server = 'khcuy9chlt.database.windows.net' 
    bdd = 'ASERTEC' 
    user = 'tecnico@khcuy9chlt'
    
    path_pass = 'data/pass.csv'
    try:
        df_pass = pd.read_csv(path_pass)
        password = df_pass.iloc[0,0]
    except:
        print('No hay fichero para cargar')
        password = 'null'
        
    fecha = '06/07/2021'

    query = 'SELECT [IDCAMPANYA] \
          ,[IDSUJETO] \
          ,[VALOR] \
          ,[DATACREACIO] \
        FROM [dbo].[T_CCT_ALISYS_DATOSCLIENTE] \
        WHERE DATACREACIO IS NOT NULL AND DATACREACIO > DATEADD(DAY,-15,\'' + fecha + '\' ) AND CLAVE = \'LLamadaTipoServicio\''
    print(query)
    conn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}',server='tcp:khcuy9chlt.database.windows.net,1433',database=bdd, uid=user, pwd=password)

    df = pd.read_sql_query(query, conn)
    
    df = df[['IDCAMPANYA','VALOR','DATACREACIO']]
    
    df['tCreacion'] = pd.to_datetime(df['DATACREACIO'])

    df['Date'] = df['tCreacion'].dt.date
    df['Date'] = df['Date'].astype('str')

    df['dateCreacion'] = df['tCreacion'].dt.date
    df['dateCreacion'] = df['dateCreacion'].astype('str')
    
    df = df[['Date','dateCreacion','IDCAMPANYA','VALOR']]
    
    df = df.groupby(['Date','IDCAMPANYA','VALOR']).count()
    
    df = df.reset_index()
    
    df = df[(df['VALOR'] == 'Estado de Pedido') | (df['VALOR'] == 'No tiene Numero')]
    
    df = df.groupby('Date').sum()[['dateCreacion']]
    
    return df
    

In [4]:
df_calls = getCallData()

SELECT [IDCAMPANYA]           ,[IDSUJETO]           ,[VALOR]           ,[DATACREACIO]         FROM [dbo].[T_CCT_ALISYS_DATOSCLIENTE]         WHERE DATACREACIO IS NOT NULL AND DATACREACIO > DATEADD(DAY,-15,'06/07/2021' ) AND CLAVE = 'LLamadaTipoServicio'


In [5]:
df_calls.head()

,dateCreacion
Date,
2021-05-24,96
2021-05-25,124
2021-05-26,95
2021-05-27,131
2021-05-28,90


# Datos Envíos

In [6]:
# Devuelve un df unificado de todos los ficheros cargados
def getDataEnvios(df):
    df = df.iloc[:, [0,4,15]]
    df['Fecha envío'] = pd.to_datetime(df['Fecha envío'], dayfirst=True)
    dias = list(calendar.day_name)
    df['dayofweek'] = [dias[numero] for numero in df['Fecha envío'].dt.dayofweek]
    df['Fecha envío'] =  pd.to_datetime(df['Fecha envío']).astype('str')
    df = df[['Fecha envío','Código servicio','dayofweek']]
    df.reset_index(inplace=True)
    df = df[['Fecha envío','Código servicio','dayofweek']]
    df['total'] = 1
    encoder = OneHotEncoder(handle_unknown='ignore')
    enc = df[['dayofweek']]
    encoder.fit(enc.values.reshape(-1,1))
    encoded = encoder.transform(enc.values.reshape(-1,1))
    encoded_df = pd.DataFrame(encoded.todense())
    encoded_df.columns = encoder.categories_[0]
    df = pd.concat([df,encoded_df], axis=1)
    df.drop(columns=['Código servicio','dayofweek'], inplace=True)
    df = df.groupby(['Fecha envío']).sum()
    df.Monday = [ 1 if value > 0 else 0 for value in df.Monday]
    df.Tuesday = [ 1 if value > 0 else 0 for value in df.Tuesday]
    df.Wednesday = [ 1 if value > 0 else 0 for value in df.Wednesday]
    df.Thursday = [ 1 if value > 0 else 0 for value in df.Thursday]
    df.Friday = [ 1 if value > 0 else 0 for value in df.Friday]
    df.Saturday = [ 1 if value > 0 else 0 for value in df.Saturday]
    df.Sunday = [ 1 if value > 0 else 0 for value in df.Sunday]
    df = df.drop(labels=['Saturday','Sunday'], axis=1)
    
    return df
    

In [7]:
df4800 = pd.read_excel('data/4800.xlsx')
df4802 = pd.read_excel('data/4802.xlsx')
df4803 = pd.read_excel('data/4803.xlsx')
df4806 = pd.read_excel('data/4806.xlsx')
df4810 = pd.read_excel('data/4810.xlsx')

c:\repos\tfm\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [8]:
df_shipping = pd.DataFrame()
df_shipping = pd.concat([df_shipping, df4800])
df_shipping = pd.concat([df_shipping, df4802])
df_shipping = pd.concat([df_shipping, df4803])
df_shipping = pd.concat([df_shipping, df4806])
df_shipping = pd.concat([df_shipping, df4810])

In [9]:
df_shippings = getDataEnvios(df_shipping)

<ipython-input-6-eece34436dc4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha envío'] = pd.to_datetime(df['Fecha envío'], dayfirst=True)
<ipython-input-6-eece34436dc4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayofweek'] = [dias[numero] for numero in df['Fecha envío'].dt.dayofweek]
<ipython-input-6-eece34436dc4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [10]:
df_shippings.head(50)

,total,Friday,Monday,Thursday,Tuesday,Wednesday
Fecha envío,,,,,,
2021-06-22,2288,0,0,0,1,0
2021-06-23,2333,0,0,0,0,1
2021-06-24,1221,0,0,1,0,0
2021-06-25,2445,1,0,0,0,0
2021-06-26,583,0,0,0,0,0
2021-06-27,372,0,0,0,0,0
2021-06-28,4317,0,1,0,0,0
2021-06-29,2688,0,0,0,1,0
2021-06-30,2711,0,0,0,0,1


# Union de los datos de llamadas 

In [11]:
def dataJoin(df_calls, df_shippins):
    df = pd.concat([df_shippins, df_calls], axis=1).reset_index()
    df['Fecha'] = df['index']
    df.drop('index', axis='columns', inplace=True)
    
    ordenado = ['Fecha', 'total', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'dateCreacion']
    df = df[ordenado]
    
    df.dropna(inplace=True)
    
    return df

In [12]:
df = dataJoin(df_calls, df_shippings)

In [13]:
df.head(20)

,Fecha,total,Monday,Tuesday,Wednesday,Thursday,Friday,dateCreacion
0,2021-06-22,2288.0,0.0,1.0,0.0,0.0,0.0,113.0
1,2021-06-23,2333.0,0.0,0.0,1.0,0.0,0.0,132.0
2,2021-06-24,1221.0,0.0,0.0,0.0,1.0,0.0,90.0
3,2021-06-25,2445.0,0.0,0.0,0.0,0.0,1.0,92.0
6,2021-06-28,4317.0,1.0,0.0,0.0,0.0,0.0,117.0
7,2021-06-29,2688.0,0.0,1.0,0.0,0.0,0.0,160.0
8,2021-06-30,2711.0,0.0,0.0,1.0,0.0,0.0,139.0
9,2021-07-01,2539.0,0.0,0.0,0.0,1.0,0.0,172.0
10,2021-07-02,2423.0,0.0,0.0,0.0,0.0,1.0,176.0
13,2021-07-05,3860.0,1.0,0.0,0.0,0.0,0.0,192.0


# Data Engineering sobre los datos de llamadas y envíos

In [14]:
def dataEngineering(df):
    # Lags de envios
    df['envioslag1'] = df['total'].shift(periods=1)
    df['envioslag6'] = df['total'].shift(periods=6)
    df['envioslag7'] = df['total'].shift(periods=7)
    df['envioslag8'] = df['total'].shift(periods=8)
    
    #Renombrar por claridad
    df['envios'] = df['total']
    df.drop('total', axis='columns', inplace=True)

    # Datos estacionales
    df['sinlag1'] = np.sin(2*np.pi*(1/1)*df.index)
    df['coslag1'] = np.cos(2*np.pi*(1/1)*df.index)

    df['sinlag2'] = np.sin(2*np.pi*(1/2)*df.index)
    df['coslag2'] = np.cos(2*np.pi*(1/2)*df.index)

    df['sinlag3'] = np.sin(2*np.pi*(1/3)*df.index)
    df['coslag3'] = np.cos(2*np.pi*(1/3)*df.index)

    df['sinlag4'] = np.sin(2*np.pi*(1/4)*df.index)
    df['coslag4'] = np.cos(2*np.pi*(1/4)*df.index)

    df['sinlag5'] = np.sin(2*np.pi*(1/5)*df.index)
    df['coslag5'] = np.cos(2*np.pi*(1/5)*df.index)

    df['sinlag6'] = np.sin(2*np.pi*(1/6)*df.index)
    df['coslag6'] = np.cos(2*np.pi*(1/6)*df.index)

    df['sinlag7'] = np.sin(2*np.pi*(1/7)*df.index)
    df['coslag7'] = np.cos(2*np.pi*(1/7)*df.index)
    
    # Lags de llamadas
    df['llamadaslag1'] = df['dateCreacion'].shift(periods=1)
    df['llamadaslag2'] = df['dateCreacion'].shift(periods=2)
    df['llamadaslag3'] = df['dateCreacion'].shift(periods=3)
    df['llamadaslag4'] = df['dateCreacion'].shift(periods=4)
    df['llamadaslag5'] = df['dateCreacion'].shift(periods=5)
    df['llamadaslag6'] = df['dateCreacion'].shift(periods=6)
    df['llamadaslag7'] = df['dateCreacion'].shift(periods=7)
    
    #Renombrar por claridad
    df['llamadas'] = df['dateCreacion']
    df.drop('dateCreacion', axis='columns', inplace=True)
    
    df.dropna(inplace=True)
    
    df  = df.tail(1).iloc[:,1:-1]
    
    df.reset_index(inplace=True)
    df.drop('index', axis='columns', inplace=True)
    
    return df

In [15]:
df = dataEngineering(df)

In [16]:
df.head()

,Monday,Tuesday,Wednesday,Thursday,Friday,envioslag1,envioslag6,envioslag7,envioslag8,envios,...,coslag6,sinlag7,coslag7,llamadaslag1,llamadaslag2,llamadaslag3,llamadaslag4,llamadaslag5,llamadaslag6,llamadaslag7
0,0.0,1.0,0.0,0.0,0.0,3860.0,4317.0,2445.0,1221.0,1909.0,...,-0.5,-4.898587e-16,1.0,192.0,176.0,172.0,139.0,160.0,117.0,92.0


# Normalizacion de datos

In [17]:
def dataScaler(df):
    # Cargar scaler desde Pickle
    scaler_pkl_filename = 'data/scaler.pkl'
    scaler_pkl = open(scaler_pkl_filename, 'rb')
    scaler = pickle.load(scaler_pkl)
    
    df = scaler.transform(df)
    
    return df

In [18]:
df = dataScaler(df)

In [19]:
df

array([[-5.27818506e-01,  1.97455727e+00, -5.06442642e-01,
        -5.02865353e-01, -4.55802844e-01,  1.43487631e+00,
         1.86778110e+00,  2.26453857e-02, -1.15961450e+00,
        -4.82754326e-01,  6.26946726e-01,  0.00000000e+00,
         2.59702606e-02,  9.72845605e-01, -1.23820240e+00,
        -7.05890776e-01,  3.32108736e-15, -1.40157309e+00,
        -1.32948540e+00,  4.45428804e-01,  1.20462378e+00,
        -7.52048617e-01, -2.86647805e-03,  1.22504454e+00,
        -1.50892862e-01, -2.95005677e-01, -3.37366181e-01,
        -6.69694554e-01, -4.63850573e-01, -8.84524473e-01,
        -1.12401443e+00]])

# Cargar modelo

In [20]:
def dataPredict(data):
    # Cargar modelo desde Pickle
    regl_pkl_filename = 'data/regl.pkl'
    regl_pkl_model = open(regl_pkl_filename, 'rb')
    regl = pickle.load(regl_pkl_model)
    
    predict = regl.predict(data)
    
    return predict

In [21]:
prediction = dataPredict(df)

In [22]:
prediction

array([178.16055885])

# Streamlit

In [23]:
%%writefile dashBoard.py

import pandas as pd
import streamlit as st
import SessionState
import datetime
import calendar
from  functionsTFG import getCallData
from  functionsTFG import getDataEnvios
from  functionsTFG import dataJoin
from  functionsTFG import dataEngineering
from  functionsTFG import dataScaler
from  functionsTFG import dataPredict
from  functionsTFG import getTendencia

#st.set_page_config(layout="wide")

def my_widget():
    global session_state
    
    if not(session_state.todas):
        st.subheader('Carga de datos')
    
    # Carga datos Agencia 4800
    if (len(session_state.df4800)==0):
        data_file_4800 = st.file_uploader("Agencia 4800",type=['xlsx'])
        if data_file_4800 is not None:
            session_state.df4800 = pd.read_excel(data_file_4800)
            print(data_file_4800.name + ' cargado')
            raise st.script_runner.RerunException(st.script_request_queue.RerunData(None))
    
    # Carga datos Agencia 4802
    if (len(session_state.df4802)==0):
        data_file_4802 = st.file_uploader("Agencia 4802",type=['xlsx'])
        if data_file_4802 is not None:
            session_state.df4802 = pd.read_excel(data_file_4802)
            print(data_file_4802.name + ' cargado')
            raise st.script_runner.RerunException(st.script_request_queue.RerunData(None))
        
    # Carga datos Agencia 4803
    if (len(session_state.df4803)==0):
        data_file_4803 = st.file_uploader("Agencia 4803",type=['xlsx'])
        if data_file_4803 is not None:
            session_state.df4803 = pd.read_excel(data_file_4803)
            print(data_file_4803.name + ' cargado')
            raise st.script_runner.RerunException(st.script_request_queue.RerunData(None))
        
    # Carga datos Agencia 4806
    if (len(session_state.df4806)==0):
        data_file_4806 = st.file_uploader("Agencia 4806",type=['xlsx'])
        if data_file_4806 is not None:
            session_state.df4806 = pd.read_excel(data_file_4806)
            print(data_file_4806.name + ' cargado')
            raise st.script_runner.RerunException(st.script_request_queue.RerunData(None))
        
    # Carga datos Agencia 4810
    if (len(session_state.df4810)==0):
        data_file_4810 = st.file_uploader("Agencia 4810",type=['xlsx'])
        if data_file_4810 is not None:
            #file_details = {"Filename":data_file_4810.name,"FileType":data_file_4810.type,"FileSize":data_file_4810.size}
            #st.write(file_details)
            session_state.df4810 = pd.read_excel(data_file_4810)
            print(data_file_4810.name + ' cargado')
            raise st.script_runner.RerunException(st.script_request_queue.RerunData(None))
            
    # Concatenar datos de las oficnas
    if (((len(session_state.df4800)>0) and (len(session_state.df4802)>0) and (len(session_state.df4803)>0) and (len(session_state.df4806)>0) and (len(session_state.df4810)>0)) and not(session_state.todas) ):
        print('Concatenando')
        session_state.todas = True
        session_state.df_shipping = pd.concat([session_state.df_shipping, session_state.df4800])
        session_state.df_shipping = pd.concat([session_state.df_shipping, session_state.df4802])
        session_state.df_shipping = pd.concat([session_state.df_shipping, session_state.df4803])
        session_state.df_shipping = pd.concat([session_state.df_shipping, session_state.df4806])
        session_state.df_shipping = pd.concat([session_state.df_shipping, session_state.df4810])
        raise st.script_runner.RerunException(st.script_request_queue.RerunData(None))
        
#########################################################################################################################################

# Variables locales
session_state = SessionState.get(df4800 = pd.DataFrame(), 
                                 df4802 = pd.DataFrame(),
                                 df4803 = pd.DataFrame(),
                                 df4806 = pd.DataFrame(),
                                 df4810 = pd.DataFrame(),
                                 df_shipping = pd.DataFrame(),
                                 todas=False)

dtoday = datetime.datetime.today()
dias = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
diasesp = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
dia = dias[dtoday.weekday()]
diaesp = diasesp[dtoday.weekday()]

st.write('# Información relativa a la campaña en el día de hoy')

df_info = pd.read_pickle('./data/df_data')
df_info = df_info[['Fecha', 'envios', 'llamadas']][df_info[dia]==1]

containerA = st.beta_container()
columnaA1, columnaA2 = containerA.beta_columns(2)

columnaA1.pyplot(getTendencia(df_info['llamadas'], 'Distribucion de llamadas en ' + diaesp, 'gray'))
columnaA2.pyplot(getTendencia(df_info['envios'], 'Distribucion de envios en ' + diaesp, 'green'))

containerB = st.beta_container()
columnaB1, columnaB2 = containerB.beta_columns(2)

columnaB1.write('##### La media de llamadas en ' + diaesp + ' es: ' + str(df_info['llamadas'].mean().round(2)) + ' y una desviación de: ' + str(df_info['llamadas'].std().round(2)) )
columnaB2.write('##### La media de envios en ' + diaesp + ' es: ' + str(df_info['envios'].mean().round(2)) + ' y una desviación de: ' + str(df_info['envios'].std().round(2))  )

if (session_state.todas):
    st.write('# Predicción llamadas contact center')
    
    df_calls = getCallData()
    df_shippings = getDataEnvios(session_state.df_shipping)
    print('Haciendo join de los datos')
    df = dataJoin(df_calls, df_shippings)
    print('Haciendo dataengineering')
    df, fecha, totalEnvios = dataEngineering(df)
    print('Normalizando datos')
    df = dataScaler(df)
    prediction = dataPredict(df)
    
    fecha = '**_ '+ str(fecha) +' _**'
    totalEnvios = '**_ '+ str(int(totalEnvios.round(0))) +' _**'
    prediction = '**_ '+ str(int(prediction.round(0))) +' _**'
    
    st.write('Para el dia ' + fecha + ' se recibirá un total de '+ totalEnvios + ' envios y se preveen un total de ' + prediction + ' llamadas')

with st.sidebar:
    clicked = my_widget()
    if (session_state.todas == True):
        st.header('Carga completa')
        st.write('Ahora ya puede cerrar esta barra de navegacion')

Overwriting dashBoard.py
